In [1]:
import sqlite3

# Connect to the SQLite database or create a new one if it doesn't exist
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

# Create the 'unclassified' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS unclassified (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        path TEXT NOT NULL,
        x INTEGER NOT NULL,
        y INTEGER NOT NULL,
        width INTEGER NOT NULL,
        height INTEGER NOT NULL
    )
''')

# Create the 'criminal' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS criminal (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        path TEXT NOT NULL,
        x INTEGER NOT NULL,
        y INTEGER NOT NULL,
        width INTEGER NOT NULL,
        height INTEGER NOT NULL
    )
''')

# Create the 'non_criminal' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS non_criminal (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        path TEXT NOT NULL,
        x INTEGER NOT NULL,
        y INTEGER NOT NULL,
        width INTEGER NOT NULL,
        height INTEGER NOT NULL
    )
''')

# Create the 'junk' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS junk (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        path TEXT NOT NULL,
        x INTEGER NOT NULL,
        y INTEGER NOT NULL,
        width INTEGER NOT NULL,
        height INTEGER NOT NULL
    )
''')

# Commit the changes and close the database connection
conn.commit()
conn.close()

In [ ]:
<?php
// Connect to the SQLite database
$database = new SQLite3('/home/runner/PHP-Web-Server/my_database.db');

if (!$database) {
    die("Error: Unable to connect to the database");
}

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    $id = $_POST['id'];
    $table = $_POST['table'];

    if ($table === 'unclassified') {
        // Handle moving an image from 'unclassified' to one of the other categories
        if (in_array($table, ['criminal', 'non_criminal', 'junk'])) {
            $query = "INSERT INTO $table SELECT * FROM unclassified WHERE id = $id";
            $database->exec($query);

            // Delete the row from the unclassified table
            $database->exec("DELETE FROM unclassified WHERE id = $id");
        }
    } else {
        // Handle moving an image from one of the other categories to 'unclassified'
        $query = "INSERT INTO unclassified SELECT * FROM $table WHERE id = $id";
        $database->exec($query);

        // Delete the row from the current table
        $database->exec("DELETE FROM $table WHERE id = $id");
    }

    // Check if the table is 'unclassified', and hide the "Unclassified" button
    if ($table === 'unclassified') {
        $unclassifiedButtonVisible = 'none';
    } else {
        $unclassifiedButtonVisible = 'inline';
    }
} else {
    $unclassifiedButtonVisible = 'none';
}

// Fetch data from the tables based on the selected table
$tableName = $_GET['table'] ?? 'unclassified';
$data = $database->query("SELECT * FROM $tableName");
?>

<!DOCTYPE html>
<html>
<head>
    <title>Image Classifier</title>
    <style>
        .container {
            display: flex;
            justify-content: space-between;
        }

        .table-container {
            width: 80%;
            height: 400px; /* Set a fixed height for the table container */
            overflow-y: auto; /* Enable vertical scrolling if needed */
        }

        table {
            border-collapse: collapse;
            width: 100%;
        }

        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }

        th {
            background-color: #f2f2f2;
        }

        tr:hover {
            background-color: #f5f5f5;
        }

        .image-container {
            flex: 1;
            text-align: center;
            display: none;
            padding-top: 80px;
            max-height: 400px; /* Set a fixed max height for the image container */
        }

        img {
            max-width: 100%;
            max-height: 100%;
        }

        /* Style for classification buttons */
        .classification-button {
            margin: 5px;
            padding: 10px;
            border: none;
            cursor: pointer;
        }

        .criminal-button {
            background-color: red;
            color: white;
        }

        .non-criminal-button {
            background-color: green;
            color: white;
        }

        .junk-button {
            background-color: gray;
            color: white;
        }

        /* Style for the "Unclassified" button */
        .unclassified-button {
            background-color: blue;
            color: white;
            display: <?php echo $unclassifiedButtonVisible; ?>; /* Show or hide the button based on the condition */
        }
    </style>
</head>
<body>
    <h1>Image Classifier</h1>

    <div class="container">
        <div class="table-container">
            <label for="tableSelect">Select Table:</label>
            <select id="tableSelect">
                <option value="unclassified" <?php if ($tableName === 'unclassified') echo 'selected'; ?>>Unclassified</option>
                <option value="criminal" <?php if ($tableName === 'criminal') echo 'selected'; ?>>Criminal</option>
                <option value="non_criminal" <?php if ($tableName === 'non_criminal') echo 'selected'; ?>>Non-Criminal</option>
                <option value="junk" <?php if ($tableName === 'junk') echo 'selected'; ?>>Junk</option>
            </select>

            <table>
                <tr>
                    <th>ID</th>
                    <th>Path</th>
                    <th>X</th>
                    <th>Y</th>
                    <th>Width</th>
                    <th>Height</th>
                </tr>
                <?php while ($row = $data->fetchArray()): ?>
                    <tr>
                        <td><?= $row['id'] ?></td>
                        <td><?= $row['path'] ?></td>
                        <td><?= $row['x'] ?></td>
                        <td><?= $row['y'] ?></td>
                        <td><?= $row['width'] ?></td>
                        <td><?= $row['height'] ?></td>
                    </tr>
                <?php endwhile; ?>
            </table>
        </div>

        <div class="image-container">
            <img id="imageToShow" src="" alt="Image">
            <form method="POST">
                <input type="hidden" id="imageId" name="id" value="">
                <?php if ($tableName === 'unclassified'): ?>
                    <button class="classification-button criminal-button" onclick="classifyImage('criminal')">Criminal</button>
                    <button class="classification-button non-criminal-button" onclick="classifyImage('non_criminal')">Non-Criminal</button>
                    <button class="classification-button junk-button" onclick="classifyImage('junk')">Junk</button>
                <?php endif; ?>
                <!-- "Unclassified" button -->
                <button class="classification-button unclassified-button" onclick="moveToUnclassified()">Unclassified</button>
            </form>
        </div>
    </div>

    <script>
        var imageIdToClassify = "";
        var classificationButtons = document.querySelectorAll('.classification-button');
        var unclassifiedButton = document.querySelector('.unclassified-button');
        var imageContainer = document.querySelector('.image-container');
        var imageToShow = document.getElementById('imageToShow');
        var imageIdField = document.getElementById('imageId');

        function classifyImage(classification) {
            var form = document.querySelector('form');
            imageIdField.value = imageIdToClassify; // Set the image ID to classify
            var input = document.createElement('input');
            input.type = "hidden";
            input.name = "table";
            input.value = classification;
            form.appendChild(input);
            form.submit();
        }

        // Function to move the image back to "Unclassified" table
        function moveToUnclassified() {
            var form = document.querySelector('form');
            var input = document.createElement('input');
            input.type = "hidden";
            input.name = "table";
            input.value = "unclassified";
            form.appendChild(input);
            form.submit();
        }

        document.addEventListener('DOMContentLoaded', function() {
            var rows = document.querySelectorAll('table tr');
            var tableSelect = document.getElementById('tableSelect');

            rows.forEach(function(row) {
                row.addEventListener('click', function() {
                    var pathCell = row.getElementsByTagName('td')[1].textContent;
                    var idCell = row.getElementsByTagName('td')[0].textContent;

                    imageToShow.src = pathCell;
                    imageIdField.value = idCell;
                    imageIdToClassify = idCell; // Set the image ID to classify
                    imageContainer.style.display = 'block';

                    // Show the "Unclassified" button when necessary
                    if (tableSelect.value !== 'unclassified') {
                        unclassifiedButton.style.display = 'inline';
                    }

                    // Enable or disable classification buttons based on the selected table
                    if (tableSelect.value === 'unclassified') {
                        classificationButtons.forEach(function(button) {
                            button.disabled = false;
                        });
                    } else {
                        classificationButtons.forEach(function(button) {
                            button.disabled = true;
                        });
                    }
                });
            });

            tableSelect.addEventListener('change', function() {
                var selectedTable = tableSelect.options[tableSelect.selectedIndex].value;
                location.href = '?table=' + selectedTable;
                imageContainer.style.display = 'none'; // Hide the image when changing the table

                // Hide the "Unclassified" button when switching tables
                unclassifiedButton.style.display = 'none';
            });
        });
    </script>
</body>
</html>

In [ ]:
if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    $id = $_POST['id'];
    $table = $_POST['table'];

    if (in_array($table, ['criminal', 'non_criminal', 'junk'])) {
        $query = "INSERT INTO $table SELECT * FROM unclassified WHERE id = $id";
        $database->exec($query);

        // Delete the row from the unclassified table
        $database->exec("DELETE FROM unclassified WHERE id = $id");
    }

    // Check if the table is 'unclassified', and hide the "Unclassified" button
    if ($table === 'unclassified') {
        $unclassifiedButtonVisible = 'none';
    } else {
        $unclassifiedButtonVisible = 'inline';
    }
} else {
    $unclassifiedButtonVisible = 'none';
}

// Fetch data from the tables based on the selected table
$tableName = $_GET['table'] ?? 'unclassified';
$data = $database->query("SELECT * FROM $tableName");
?>

In [2]:
import cv2
import os
import sqlite3

# Load cascade classifier training file for haarcascade
haar_face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')

def detect_face(f_cascade, colored_img, scaleFactor=1.1):
    gray = cv2.cvtColor(colored_img, cv2.COLOR_BGR2GRAY)
    faces = f_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=5)

    if len(faces) == 1:
        x, y, w, h = faces[0]
        # Draw a rectangle around the detected face
        cv2.rectangle(colored_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        return colored_img, x, y, w, h
    else:
        return None

def process_image(original_folder, cropped_folder, filename, conn):
    image_path = os.path.join(original_folder, filename)
    img = cv2.imread(image_path)
    
    # Resize the image to 256x256
    img = cv2.resize(img, (256, 256))

    result = detect_face(haar_face_cascade, img)

    if result is not None:
        img_with_rectangles, x, y, w, h = result

        # Save the image with rectangles in the existing cropped folder
        save_path = os.path.join(cropped_folder, filename)
        cv2.imwrite(save_path, img_with_rectangles)

        # Convert x, y, width, and height to integers
        x, y, w, h = int(x), int(y), int(w), int(h)

        # Insert values into the database (assuming 'unallocated' table exists)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO unclassified (path, x, y, width, height) VALUES (?, ?, ?, ?, ?)",
                       (save_path, x, y, w, h))
        conn.commit()
    else:
        print(f"No face detected in {filename}")

    # os.remove(image_path)  # You can uncomment this line to delete the original image.

def process_images_in_folder(original_folder, cropped_folder, conn):
    for filename in os.listdir(original_folder):
        if filename.endswith(".jpg"):
            process_image(original_folder, cropped_folder, filename, conn)

# Specify the directories for original and cropped images
original_folder = './original/'
cropped_folder = './cropped/'

# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')

# Call the function to process and draw rectangles around faces in the directory and store values in the 'unallocated' table
process_images_in_folder(original_folder, cropped_folder, conn)

# Close the database connection
conn.close()

No face detected in 8.jpg
No face detected in 15.jpg
No face detected in 23.jpg
No face detected in 2.jpg


In [16]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

# List of table names
table_names = ['unclassified', 'criminal', 'non_criminal', 'junk']

# Iterate through the table names and fetch data
for table_name in table_names:
    print(f"Data in the '{table_name}' table:")
    cursor.execute(f"SELECT * FROM {table_name}")
    
    # Fetch all rows from the table
    rows = cursor.fetchall()
    
    # Print the table header
    print("ID | Path | x | y | width | height")
    
    # Print the data in each row
    for row in rows:
        print(row)

    print("\n")

# Close the database connection
conn.close()

Data in the 'unclassified' table:
ID | Path | x | y | width | height
(1, './cropped/9.jpg', 172, 113, 44, 44)
(2, './cropped/12.jpg', 65, 46, 106, 106)
(3, './cropped/13.jpg', 70, 63, 119, 119)
(4, './cropped/11.jpg', 140, 26, 40, 40)
(5, './cropped/10.jpg', 59, 49, 114, 114)
(6, './cropped/4.jpg', 86, 15, 51, 51)
(7, './cropped/5.jpg', 100, 13, 46, 46)
(8, './cropped/7.jpg', 70, 56, 133, 133)
(9, './cropped/6.jpg', 62, 52, 137, 137)
(10, './cropped/3.jpg', 34, 33, 165, 165)
(11, './cropped/1.jpg', 65, 46, 106, 106)


Data in the 'criminal' table:
ID | Path | x | y | width | height


Data in the 'non_criminal' table:
ID | Path | x | y | width | height


Data in the 'junk' table:
ID | Path | x | y | width | height




In [ ]:
<?php
// Connect to the SQLite database
$database = new SQLite3('/home/runner/PHP-Web-Server/my_database.db');

if (!$database) {
    die("Error: Unable to connect to the database");
}

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    $id = $_POST['id'];
    $table = $_POST['table'];

    if (in_array($table, ['criminal', 'non_criminal', 'junk'])) {
        $query = "INSERT INTO $table SELECT * FROM unclassified WHERE id = $id";
        $database->exec($query);

        // Delete the row from the unclassified table
        $database->exec("DELETE FROM unclassified WHERE id = $id");
    }
}

// Fetch data from the tables based on the selected table
$tableName = $_GET['table'] ?? 'unclassified';
$data = $database->query("SELECT * FROM $tableName");
?>

<!DOCTYPE html>
<html>
<head>
    <title>Image Classifier</title>
    <style>
        .container {
            display: flex;
            justify-content: space-between;
        }

        .table-container {
            width: 80%;
            height: 400px; /* Set a fixed height for the table container */
            overflow-y: auto; /* Enable vertical scrolling if needed */
        }

        table {
            border-collapse: collapse;
            width: 100%;
        }

        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }

        th {
            background-color: #f2f2f2;
        }

        tr:hover {
            background-color: #f5f5f5;
        }

        .image-container {
            flex: 1;
            text-align: center;
            display: none;
            padding-top: 80px;
            max-height: 400px; /* Set a fixed max height for the image container */
        }

        img {
            max-width: 100%;
            max-height: 100%;
        }

        /* Style for classification buttons */
        .classification-button {
            margin: 5px;
            padding: 10px;
            border: none;
            cursor: pointer;
        }

        .criminal-button {
            background-color: red;
            color: white;
        }

        .non-criminal-button {
            background-color: green;
            color: white;
        }

        .junk-button {
            background-color: gray;
            color: white;
        }
    </style>
</head>
<body>
    <h1>Image Classifier</h1>

    <div class="container">
        <div class="table-container">
            <label for="tableSelect">Select Table:</label>
            <select id="tableSelect">
                <option value="unclassified" <?php if ($tableName === 'unclassified') echo 'selected'; ?>>Unclassified</option>
                <option value="criminal" <?php if ($tableName === 'criminal') echo 'selected'; ?>>Criminal</option>
                <option value="non_criminal" <?php if ($tableName === 'non_criminal') echo 'selected'; ?>>Non-Criminal</option>
                <option value="junk" <?php if ($tableName === 'junk') echo 'selected'; ?>>Junk</option>
            </select>

            <table>
                <tr>
                    <th>ID</th>
                    <th>Path</th>
                    <th>X</th>
                    <th>Y</th>
                    <th>Width</th>
                    <th>Height</th>
                </tr>
                <?php while ($row = $data->fetchArray()): ?>
                    <tr>
                        <td><?= $row['id'] ?></td>
                        <td><?= $row['path'] ?></td>
                        <td><?= $row['x'] ?></td>
                        <td><?= $row['y'] ?></td>
                        <td><?= $row['width'] ?></td>
                        <td><?= $row['height'] ?></td>
                    </tr>
                <?php endwhile; ?>
            </table>
        </div>

        <div class="image-container">
            <img id="imageToShow" src="" alt="Image">
            <form method="POST">
                <input type="hidden" id="imageId" name="id" value="">
                <?php if ($tableName === 'unclassified'): ?>
                    <button class="classification-button criminal-button" onclick="classifyImage('criminal')">Criminal</button>
                    <button class="classification-button non-criminal-button" onclick="classifyImage('non_criminal')">Non-Criminal</button>
                    <button class="classification-button junk-button" onclick="classifyImage('junk')">Junk</button>
                <?php endif; ?>
            </form>
        </div>
    </div>

    <script>
        var imageIdToClassify = "";
        var classificationButtons = document.querySelectorAll('.classification-button');
        var imageContainer = document.querySelector('.image-container');
        var imageToShow = document.getElementById('imageToShow');
        var imageIdField = document.getElementById('imageId');

        function classifyImage(classification) {
            var form = document.querySelector('form');
            imageIdField.value = imageIdToClassify; // Set the image ID to classify
            var input = document.createElement('input');
            input.type = "hidden";
            input.name = "table";
            input.value = classification;
            form.appendChild(input);
            form.submit();
        }

        document.addEventListener('DOMContentLoaded', function() {
            var rows = document.querySelectorAll('table tr');
            var tableSelect = document.getElementById('tableSelect');

            rows.forEach(function(row) {
                row.addEventListener('click', function() {
                    var pathCell = row.getElementsByTagName('td')[1].textContent;
                    var idCell = row.getElementsByTagName('td')[0].textContent;

                    imageToShow.src = pathCell;
                    imageIdField.value = idCell;
                    imageIdToClassify = idCell; // Set the image ID to classify
                    imageContainer.style.display = 'block';

                    // Enable or disable classification buttons based on the selected table
                    if (tableSelect.value === 'unclassified') {
                        classificationButtons.forEach(function(button) {
                            button.disabled = false;
                        });
                    } else {
                        classificationButtons.forEach(function(button) {
                            button.disabled = true;
                        });
                    }
                });
            });

            tableSelect.addEventListener('change', function() {
                var selectedTable = tableSelect.options[tableSelect.selectedIndex].value;
                location.href = '?table=' + selectedTable;
                imageContainer.style.display = 'none'; // Hide the image when changing the table
            });
        });
    </script>
</body>
</html>